In [2]:
import ee
import pandas as pd
import geemap
import json
import math


In [ ]:

ee.Authenticate()
ee.Initialize(project="maps-disaster")


Export started. Check your Google Drive when done.


In [ ]:

gaul1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
south_sulawesi = gaul1.filter(
    ee.Filter.And(
        ee.Filter.eq("ADM0_NAME", "Indonesia"),
        ee.Filter.eq("ADM1_NAME", "Sulawesi Selatan"),
    )
)


In [ ]:


flood_events = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1").filterBounds(
    south_sulawesi
)



In [ ]:


def get_date_vars(image):
    date = ee.Date(image.get("system:time_start"))
    return {
        "date": date,
        "date_str": date.format("YYYY-MM-dd"),
        "year": date.get("year"),
    }


def get_flood_labels(image):
    return image.select(["flooded", "jrc_perm_water"])


def masked_img(name):
    return ee.Image.constant(0).rename(name).updateMask(ee.Image.constant(0))


def safe_sum(col, name):
    summed = col.sum().rename(name)
    return ee.Image(ee.Algorithms.If(col.size().gt(0), summed, masked_img(name)))


def safe_mean(col, name, fn=None):
    if fn:
        col = col.map(fn)
    mean_img = col.mean().rename(name)
    return ee.Image(ee.Algorithms.If(col.size().gt(0), mean_img, masked_img(name)))



In [ ]:

def get_precip(date):
    c1 = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(
        date.advance(-1, "day"), date
    )
    c3 = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(
        date.advance(-3, "day"), date
    )
    return safe_sum(c1, "precip_1d"), safe_sum(c3, "precip_3d")


def get_indices(date):
    ndvi_col = (
        ee.ImageCollection("MODIS/061/MOD13A2")
        .filterDate(date.advance(-16, "day"), date.advance(16, "day"))
        .select("NDVI")
    )
    ndwi_col = (
        ee.ImageCollection("MODIS/061/MOD09GA")
        .filterDate(date.advance(-16, "day"), date.advance(16, "day"))
        .map(lambda i: i.normalizedDifference(["sur_refl_b02", "sur_refl_b04"]))
    )
    ndvi = safe_mean(ndvi_col, "NDVI")
    ndwi = safe_mean(ndwi_col, "NDWI")
    return ndvi, ndwi


def get_water_occurrence():
    return (
        ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
        .select("occurrence")
        .rename("water_occurrence")
    )


def get_lst(date):
    col = (
        ee.ImageCollection("MODIS/061/MOD11A2")
        .filterDate(date, date.advance(1, "day"))
        .select("LST_Day_1km")
    )
    return ee.Image(
        ee.Algorithms.If(
            col.size().gt(0),
            col.first().multiply(0.02).rename("LST_day"),
            masked_img("LST_day"),
        )
    )




def get_landcover(year):
    img = (
        ee.ImageCollection("MODIS/061/MCD12Q1")
        .filter(ee.Filter.calendarRange(year, year, "year"))
        .first()
    )
    return img.select("LC_Type1").rename("landcover")


def get_terrain():
    srtm = ee.Image("USGS/SRTMGL1_003")
    return (
        srtm.select("elevation"),
        ee.Terrain.slope(srtm).rename("slope"),
        ee.Terrain.aspect(srtm).rename("aspect"),
    )


def get_hydro_topo(slope):
    merit = ee.Image("MERIT/Hydro/v1_0_1")
    ups = merit.select("upa").rename("upstream_area")
    slope_r = slope.multiply(math.pi / 180)
    twi = ups.divide(slope_r.tan().max(ee.Image.constant(0.001))).log().rename("TWI")
    return ups, twi




In [ ]:


def extract_features(image):
    dv = get_date_vars(image)
    fb = get_flood_labels(image)
    p1, p3 = get_precip(dv["date"])
    ndvi, ndwi = get_indices(dv["date"])
    wo = get_water_occurrence()
    lst = get_lst(dv["date"])
    lc = get_landcover(dv["year"])
    elev, slope, aspect = get_terrain()
    ups, twi = get_hydro_topo(slope)

    feat = fb.addBands(
        [p1, p3, ndvi, ndwi, wo, lst, lc, elev, slope, aspect, ups, twi]
    )

    samples = feat.sample(
        region=south_sulawesi.geometry(),
        scale=250,
        projection="EPSG:4326",
        geometries=True,
        tileScale=8,
        dropNulls=False,
        numPixels=500_000,  # ← cap pixels per event
    ).map(
        lambda f: f.set(
            {
                "date": dv["date_str"],
                "lon": f.geometry().coordinates().get(0),
                "lat": f.geometry().coordinates().get(1),
            }
        )
    )

    return samples




In [ ]:

all_features = flood_events.map(extract_features).flatten()
task = ee.batch.Export.table.toDrive(
    collection=all_features,
    description="modis_flood_features_full_latest",
    fileFormat="CSV",
    selectors=[
        "date", "lon", "lat",
        "flooded", "jrc_perm_water",
        "precip_1d", "precip_3d",
        "NDVI", "NDWI",
        "water_occurrence",
        "LST_day", "soil_moisture",
        "landcover",
        "elevation", "slope", "aspect",
        "upstream_area", "TWI"
    ]
)
task.start()
print("Export started. Check your Google Drive when done.")